# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [9]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [8]:
extracted = spark.read.csv("ExtractedTweets.csv", header=True)
handles = spark.read.csv("TwitterHandles.csv", header=True)

In [9]:
extracted.show()

+--------------------+-------------+--------------------+
|               Party|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         NULL|                NULL|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
|            Democrat|RepDarrenSoto|RT @Vegalteno: Hu...|
|            Democrat|RepDarrenSoto|RT @EmgageActionF...|
|            Democrat|RepDarrenSoto|Hurricane Maria l...|
|            Democrat|RepDarrenSoto|RT @Tharryry: I a...|
|            Democrat|RepDarrenSoto|RT @HispanicCaucu...|
|            Democrat|RepDarrenSoto|RT @RepStephMurph...|
|            Democrat|RepDarrenSoto|RT @AllSaints_FL:...|
|            Democrat|RepDarrenSoto|.@realDonaldTrump...|
|            Democrat|RepDarrenSoto|Thank you to my m...|
|            D

**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [10]:
extracted.show(truncate=False)

+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                                               |Handle       |Tweet                                                                                                                                       |
+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                                            |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                                            |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized 

In [11]:
handles.show(truncate=False)

+--------+--------------------+---------------+---------------------------------------------------------------------------+
|Party   |Name                |Handle         |AvatarURL                                                                  |
+--------+--------------------+---------------+---------------------------------------------------------------------------+
|Democrat|US Rep. Darren Soto |RepDarrenSoto  |https://pbs.twimg.com/profile_images/824454906446745601/DtndIYe0_bigger.jpg|
|Democrat|Rep. Jacky Rosen    |RepJackyRosen  |https://pbs.twimg.com/profile_images/837772241077170177/VOpyOjAj_bigger.jpg|
|Democrat|US Rep. Al Lawson Jr|RepAlLawsonJr  |https://pbs.twimg.com/profile_images/818493713991213056/U3FMB3nB_bigger.jpg|
|Democrat|Adriano Espaillat   |RepEspaillat   |https://pbs.twimg.com/profile_images/827580972442988545/RJ5H4gCN_bigger.jpg|
|Democrat|US Rep. Darren Soto |RepDarrenSoto  |https://pbs.twimg.com/profile_images/824454906446745601/DtndIYe0_normal.jpg|
|Democra

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [12]:
extracted.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



In [13]:
handles.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- AvatarURL: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [14]:
import pyspark.sql.functions as func
# df = extracted.withColumn("Latino_Mentions", regexp.extract(extracted.Tweet,(.)(@LatinoLenader)(.),2))
clean = extracted.withColumn("Latino_Mentions", func.regexp_extract(extracted["Party"], r'(@LatinoLenader)', 1))

In [15]:
clean.show(truncate=False)

+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Party                                               |Handle       |Tweet                                                                                                                                       |Latino_Mentions|
+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Democrat                                            |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |               |
|Democrat                                            |RepDarrenSoto|RT @WinterHavenSun: Winter H

In [16]:
counts = clean.groupBy("Party").count()

In [17]:
counts.show()

+--------------------+-----+
|               Party|count|
+--------------------+-----+
|        investigat…"|    1|
|Pruitt’s Chief Of...|    1|
|05/03/… https://t...|    1|
|NOTE that… https:...|    1|
|MC Lyte - Poor Ge...|    1|
|   Start- November 1|    1|
|To all who are ce...|    1|
|The Kennedy famil...|    1|
|Glad… https://t.c...|    1|
|The U.S. should b...|    1|
|   -1st: Rosa Kirgis|    1|
|             If you:|    1|
|            As a mom|    1|
|          @jmeacham |    1|
|    #SuicidePreven…"|    1|
|                 1…"|    1|
|https://t.co/XL5w...|    1|
|I’m sure you’re j...|    1|
|As… https://t.co/...|    1|
|      ☔ Venice Beach|    1|
+--------------------+-----+
only showing top 20 rows



In [18]:
from pyspark.sql.functions import desc 
counts.orderBy(desc("count")).show()

+--------------------+-----+
|               Party|count|
+--------------------+-----+
|          Republican|44392|
|            Democrat|42068|
|            That’s…"|   28|
|https://t.co/oc6J...|   22|
|                 Now|   17|
|               Today|   13|
|         https://t…"|   12|
|              http…"|   12|
|                 h…"|   12|
|               ❌ E…"|   11|
|❌ Abandoned Hispa...|   11|
|   ❌ Passed #TaxScam|   11|
|  ❌ Terminated #DACA|   11|
|        #mepolitics"|   11|
|https://t.co/8htz...|    9|
|      https://t.co…"|    9|
|💻 Website: https...|    9|
|This. Has. To. St...|    8|
|                   4|    7|
|Semper Fidelis. h...|    7|
+--------------------+-----+
only showing top 20 rows



## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [19]:
from pyspark.sql.functions import when 
clean = extracted.withColumn("Party",
        when(extracted.Party == "Democrat", 'Democrat') 
        .when(extracted.Party == "Republican", 'Republican') 
         .otherwise('Other'))

In [20]:
clean.show()

+--------+-------------+--------------------+
|   Party|       Handle|               Tweet|
+--------+-------------+--------------------+
|Democrat|RepDarrenSoto|Today, Senate Dem...|
|Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|   Other|         NULL|                NULL|
|Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
|Democrat|RepDarrenSoto|RT @Vegalteno: Hu...|
|Democrat|RepDarrenSoto|RT @EmgageActionF...|
|Democrat|RepDarrenSoto|Hurricane Maria l...|
|Democrat|RepDarrenSoto|RT @Tharryry: I a...|
|Democrat|RepDarrenSoto|RT @HispanicCaucu...|
|Democrat|RepDarrenSoto|RT @RepStephMurph...|
|Democrat|RepDarrenSoto|RT @AllSaints_FL:...|
|Democrat|RepDarrenSoto|.@realDonaldTrump...|
|Democrat|RepDarrenSoto|Thank you to my m...|
|Democrat|RepDarrenSoto|We paid our respe...|
|   Other|         NULL|                NULL|
|Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|Democrat|RepDarrenSoto|Meet 12 incredibl...|
|Democrat|RepDarrenSoto|RT @wildli

## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [21]:
test = extracted.withColumn("clean", func.regexp_replace("Tweet", "https:", ""))

In [22]:
test.select("clean").show()

+--------------------+
|               clean|
+--------------------+
|Today, Senate Dem...|
|RT @WinterHavenSu...|
|RT @NBCLatino: .@...|
|                NULL|
|RT @NALCABPolicy:...|
|RT @Vegalteno: Hu...|
|RT @EmgageActionF...|
|Hurricane Maria l...|
|RT @Tharryry: I a...|
|RT @HispanicCaucu...|
|RT @RepStephMurph...|
|RT @AllSaints_FL:...|
|.@realDonaldTrump...|
|Thank you to my m...|
|We paid our respe...|
|                NULL|
|RT @WinterHavenSu...|
|Meet 12 incredibl...|
|RT @wildlifeactio...|
|RT @CHeathWFTV: K...|
+--------------------+
only showing top 20 rows



## 4. Remove any leading or trailing white space in the tweet column

In [23]:
from pyspark.sql.functions import trim
df2 = clean.withColumn("Tweet", trim(clean.Tweet))

In [24]:
df2.show(truncate=False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|Oth

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [25]:
df2.withColumnRenamed("Party", "Dem_Rep").show()

+--------+-------------+--------------------+
| Dem_Rep|       Handle|               Tweet|
+--------+-------------+--------------------+
|Democrat|RepDarrenSoto|Today, Senate Dem...|
|Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|   Other|         NULL|                NULL|
|Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
|Democrat|RepDarrenSoto|RT @Vegalteno: Hu...|
|Democrat|RepDarrenSoto|RT @EmgageActionF...|
|Democrat|RepDarrenSoto|Hurricane Maria l...|
|Democrat|RepDarrenSoto|RT @Tharryry: I a...|
|Democrat|RepDarrenSoto|RT @HispanicCaucu...|
|Democrat|RepDarrenSoto|RT @RepStephMurph...|
|Democrat|RepDarrenSoto|RT @AllSaints_FL:...|
|Democrat|RepDarrenSoto|.@realDonaldTrump...|
|Democrat|RepDarrenSoto|Thank you to my m...|
|Democrat|RepDarrenSoto|We paid our respe...|
|   Other|         NULL|                NULL|
|Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|Democrat|RepDarrenSoto|Meet 12 incredibl...|
|Democrat|RepDarrenSoto|RT @wildli

## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [26]:
newDF = df2.selectExpr("Party", "Handle", "CONCAT(Party, ' ', Handle) AS combined") 

In [27]:
newDF.show()

+--------+-------------+--------------------+
|   Party|       Handle|            combined|
+--------+-------------+--------------------+
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|   Other|         NULL|                NULL|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|   Other|         NULL|                NULL|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat RepDarre...|
|Democrat|RepDarrenSoto|Democrat R

In [28]:
from pyspark.sql.functions import concat_ws
newerDF = df2.select(concat_ws(" ", df2.Party, df2.Handle).alias("concatenated"))

In [29]:
newerDF.show(truncate=False)

+----------------------+
|concatenated          |
+----------------------+
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Other                 |
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Other                 |
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
|Democrat RepDarrenSoto|
+----------------------+
only showing top 20 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [30]:
import pyspark.sql.functions as func
extracted2 = spark.read.csv("ExtractedTweets.csv", header=True)
challengeQuestion = extracted2.withColumn("Hashtags", func.regexp_extract(extracted2["Tweet"], r'(#)', 1))

In [31]:
challengeQuestion.show(truncate=False)

+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Party                                               |Handle       |Tweet                                                                                                                                       |Hashtags|
+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Democrat                                            |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |#       |
|Democrat                                            |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista t

# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [10]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [11]:
from pyspark.sql.functions import col
df4 = df.withColumn("dob",col("dob").cast(DateType())).withColumn("visit1",col("visit1").cast(DateType())) \
.withColumn("visit2",col("visit2").cast(DateType())).withColumn("visit3",col("visit3").cast(DateType()))
print(df4.printSchema())

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)

None


## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [19]:
from pyspark.sql.functions import *

df4.withColumn('DIFF1', floor(datediff(df4['visit2'], df4['visit1']))).show()
df4.withColumn('DIFF2', floor(datediff(df4['visit3'], df4['visit2']))).show()

+----------+---------+----------+----------+----------+----------+-----+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|DIFF1|
+----------+---------+----------+----------+----------+----------+-----+
|  Mohammed|   Alfasy|1987-04-08|2016-01-07|2017-02-03|2018-03-02|  393|
|     Marcy|Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|  727|
|     Ginny|   Ginger|1986-07-10|2014-08-07|2015-02-03|2016-03-02|  180|
|     Vijay| Doberson|1988-05-02|2016-01-07|2018-02-03|2018-03-02|  758|
|     Orhan|  Gelicek|1987-05-11|2016-05-07|2017-01-03|2018-09-02|  241|
|     Sarah|    Jones|1956-07-06|2016-04-07|2017-08-03|2018-10-02|  483|
|      John|  Johnson|2017-10-12|2018-01-02|2018-10-03|2018-03-02|  274|
+----------+---------+----------+----------+----------+----------+-----+

+----------+---------+----------+----------+----------+----------+-----+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|DIFF2|
+----------+---------+----------+----------+------

## 8. Can you calculate the age of each patient?

In [58]:
from pyspark.sql.functions import *

ageDF = df4.withColumn('dob', floor(datediff(current_date(), df4['dob'])/365))
ageDF.show()

+----------+---------+---+----------+----------+----------+
|first_name|last_name|dob|    visit1|    visit2|    visit3|
+----------+---------+---+----------+----------+----------+
|  Mohammed|   Alfasy| 36|2016-01-07|2017-02-03|2018-03-02|
|     Marcy|Wellmaker| 37|2015-01-07|2017-01-03|2018-01-02|
|     Ginny|   Ginger| 37|2014-08-07|2015-02-03|2016-03-02|
|     Vijay| Doberson| 35|2016-01-07|2018-02-03|2018-03-02|
|     Orhan|  Gelicek| 36|2016-05-07|2017-01-03|2018-09-02|
|     Sarah|    Jones| 67|2016-04-07|2017-08-03|2018-10-02|
|      John|  Johnson|  6|2018-01-02|2018-10-03|2018-03-02|
+----------+---------+---+----------+----------+----------+



In [ ]:
# df4.createOrReplaceTempView("res")
# sqlDF = spark.sql("SELECT EXTRACT(year from `dob`) FROM res ")
# sqlDF.show()

## 9. Can you extract the month from the first visit column and call it "Month"?

In [61]:
from pyspark.sql.functions import *
monthDF = df4.withColumn('visit1', month(df4['visit1']))
monthDF.show()

+----------+---------+----------+------+----------+----------+
|first_name|last_name|       dob|visit1|    visit2|    visit3|
+----------+---------+----------+------+----------+----------+
|  Mohammed|   Alfasy|1987-04-08|     1|2017-02-03|2018-03-02|
|     Marcy|Wellmaker|1986-04-08|     1|2017-01-03|2018-01-02|
|     Ginny|   Ginger|1986-07-10|     8|2015-02-03|2016-03-02|
|     Vijay| Doberson|1988-05-02|     1|2018-02-03|2018-03-02|
|     Orhan|  Gelicek|1987-05-11|     5|2017-01-03|2018-09-02|
|     Sarah|    Jones|1956-07-06|     4|2017-08-03|2018-10-02|
|      John|  Johnson|2017-10-12|     1|2018-10-03|2018-03-02|
+----------+---------+----------+------+----------+----------+



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
newspark = SparkSession.builder.getOrCreate()
supermarket = newspark.read.csv("supermarket_sales - Sheet1.csv", header=True)
supermarket.show(truncate=True)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-----+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date| Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|01/05/2019|13:08|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22|03/08/2019|10:29|       Cash|  76.4|            4.761904762|        3.82|   9.6|
|631-41-3108|     A|   Yangon|       Normal|  Male

In [3]:
supermarket.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Tax 5%: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: string (nullable = true)
 |-- gross margin percentage: string (nullable = true)
 |-- gross income: string (nullable = true)
 |-- Rating: string (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [4]:
# new_supermarket = supermarket.withColumn('Date',col('Date').cast(DateType())) cast khong hieu qua neu muon chuyen tu string sang date
# new_supermarket = supermarket.withColumn("Date",to_date(supermarket['Date']))
# new_supermarket.show()
from pyspark.sql.functions import to_date, col, when

# Assuming 'supermarket' is your DataFrame containing the data
new_supermarket = supermarket.withColumn("Date", when(col("Date").cast("date").isNotNull(), col("Date").cast("date")).otherwise(to_date(col("Date"))))

new_supermarket.show()


+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----+-----+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|Date| Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|NULL|13:08|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22|NULL|10:29|       Cash|  76.4|            4.761904762|        3.82|   9.6|
|631-41-3108|     A|   Yangon|       Normal|  Male|  Home and lifestyle|     46.

In [5]:
new_supermarket.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Tax 5%: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: string (nullable = true)
 |-- gross margin percentage: string (nullable = true)
 |-- gross income: string (nullable = true)
 |-- Rating: string (nullable = true)



### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [6]:
from pyspark.sql.functions import *
supermarket_monthDF = new_supermarket.withColumn('Month', month(new_supermarket['Date']))
supermarket_monthDF.select(col('Month')).show()

+-----+
|Month|
+-----+
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
| NULL|
+-----+
only showing top 20 rows



## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [22]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [23]:
from pyspark.sql.functions import col, lower, trim, regexp_replace, split

# Remove punctuation, lower case, and trim whitespace
clean_reviews = reviews.withColumn("clean_review_txt", trim(lower(regexp_replace(col("review_txt"), "[^a-zA-Z0-9\\s]", ""))))

# Split the cleaned text into an array of words
clean_reviews = clean_reviews.withColumn("review_words", split(col("clean_review_txt"), "\\s+"))

clean_reviews.select("rating", "review_words").show(truncate=False)


+------+---------------------------------------------------------------------------------------------------+
|rating|review_words                                                                                       |
+------+---------------------------------------------------------------------------------------------------+
|5     |[epic, this, is, the, best, movie, i, have, ever, seen]                                            |
|4     |[pretty, good, but, i, would, have, liked, to, seen, better, special, effects]                     |
|3     |[so, so, casting, could, have, been, improved]                                                     |
|5     |[the, most, epic, movie, of, the, year, casting, was, awesome, special, effects, were, so, intense]|
|4     |[solid, but, i, would, have, liked, to, see, more, of, the, love, story]                           |
|5     |[the, bomb]                                                                                        |
+------+-----------

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [ ]:
from pyspark.sql.functions import array_contains

# Filter reviews containing the word 'Epic'
epic_reviews = clean_reviews.filter(array_contains(col("review_words"), "epic"))

# Show the filtered reviews
epic_reviews.select("rating", "review_txt").show(truncate=False)


### That's it! Great Job!